In [1]:
import pandas as pd
import numpy as np
#from post_tag import *
import csv

In [2]:
translations = pd.read_csv("training.txt", sep="\t", header=None, names=['labels','phrases'], quoting=csv.QUOTE_NONE)
translations

,labels,phrases
0,1,"La noticia de WhatsApp , aplicación de mensaje..."
1,0,"En cualquier caso , aunque la respeto mucho Fr..."
2,0,Entre los 320 individuos de actuación son el m...
3,1,Tal vez ella vaya a tomar aunque sea alguno de...
4,1,Ella ha hecho mucho por las mujeres en la músi...
5,0,"Astro Teller tiene su trabajo ideal , actualme..."
6,0,Nueve de los socios de Accel han realizado inv...
7,1,""" Otras veces , parecen guíarse de Hollywood ."
8,0,Apuestas JayBird En seguimiento inteligente pa...
9,1,"En esta ocasión , ¡ nos hemos dado cuenta !"


In [3]:
#Notes about the data:
# it's about 50/50 labels



In [4]:
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import sklearn
#from stop_words import get_stop_words
from time import time
from nltk.corpus import stopwords


In [5]:
es_stop = stopwords.words('spanish')

In [6]:
M_doc_freq = 0.80
min_doc_freq = 10

CV = CountVectorizer(max_df=M_doc_freq, min_df=min_doc_freq, stop_words=es_stop, binary=True, lowercase=True)

In [7]:
# Fit the vectorizer
t0 = time()
CV = CV.fit(translations['phrases'].values)

phrase_matrix = CV.transform(translations['phrases'].values)
print("done in %0.3fs." % (time() - t0))
tf_feature_names = CV.get_feature_names()

/Library/Python/2.7/site-packages/numpy-1.11.0rc1-py2.7-macosx-10.10-intel.egg/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


done in 1.356s.


In [8]:
n_wrds=len(CV.vocabulary_)

In [9]:
cols = ['bw' + str(i) for i in range(n_wrds)]

BofW_feats = pd.DataFrame(phrase_matrix.todense(), columns=cols)
BofW_feats.to_csv('BoW_feats.csv')

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import csv

#Generic utils
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

In [11]:
data = phrase_matrix
target = translations['labels'].values
classf =  SVC()

In [ ]:
t0 = time()
classf.fit(data,target)
print "Score of SVM: " + str(classf.score(data,target))
print("done in %0.3fs." % (time() - t0))

In [17]:
data = phrase_matrix
t0 = time()
svd = TruncatedSVD(n_components=200, random_state=42)
svd.fit(data) 
data = svd.transform(data)
print("SVD fit done in %0.3fs." % (time() - t0))
cols = ['bw' + str(i) for i in range(200)]
BofW_SVD_feats = pd.DataFrame(data, columns = cols)
BofW_SVD_feats.to_csv('BofW_SVD_feats.csv')

SVD fit done in 1.868s.


In [ ]:
t0 = time()
data = data
pca = PCA(n_components =500)
pca.fit(data)
data = pca.transform(data)
print("PCA fit done in %0.3fs." % (time() - t0))

In [ ]:
t0 = time()
classf = SVC()
scores = cross_validation.cross_val_score(classf,data,target, cv = 10)
print "Scores: " + str(scores)
print "Crossval Final score: "+ str(sum(scores)/10)
print("Cross val done in %0.3fs." % (time() - t0))

In [14]:
pipeline = Pipeline([
    ('svd',TruncatedSVD()),
    ('svm', SVC())
])
# Fine tune parameters using exaustive GridSearch:

parameters = {
    'svd__n_components': (50,100,200,300,1000),
    }
    
#done in 2334.428s
#()
#Best score: 0.513
#Best parameters set:
#	svd__n_components: 200
grid_search = GridSearchCV(pipeline, parameters ,verbose=1)

In [15]:
data = phrase_matrix

t0 = time()
grid_search.fit(data, target)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 36.5min finished


done in 2334.428s
()
Best score: 0.513
Best parameters set:
	svd__n_components: 200


In [ ]:
# How to process a test set and predict the labels 

test = pd.read_csv("example_test.txt", sep="\t", header=None, names=['labels','phrases'], lineterminator='\n', quoting=csv.QUOTE_NONE)
test

In [ ]:
testlabs = classf.predict(svd.transform(CV.transform(test['phrases'])))

In [ ]:
test.labels = test['labels'] = testlabs
test

In [ ]:
test.to_csv("example_test_output.txt", sep="\t", header=False, index=False, quoting=csv.QUOTE_NONE)

In [ ]:
test.iloc[0]['phrases']

In [ ]:
import pandas as pd
import numpy as np
from post_tag import *
import csv
from time import time
translations = pd.read_csv("training.txt", sep="\t", header=None, names=['labels','phrases'], quoting=csv.QUOTE_NONE)



In [ ]:
# play around with pos tagging
trans = translations['phrases'].apply(postag)

In [ ]:
test = pd.read_csv("example_test.txt", sep="\t", header=None, names=['labels','phrases'], lineterminator='\n', quoting=csv.QUOTE_NONE)

test

In [ ]:
t0 = time()
test_tagged = test['phrases'].apply(postag)
print("done in %0.3fs."%(time()-t0))

In [ ]:
fdist = itertools.permutations(words, 2)
permu_names = [f for f in fdist]
tags_DF = pd.DataFrame(np.asmatrix(list(test_tagged)),columns = permu_names)

test_new = pd.concat([test, tags_DF], axis=1,join_axes=[test.index])
test_new.drop('phrases',axis=1, inplace=True)
test_new.to_csv(path_or_buf='pos_tags_train_data.csv')

In [ ]:
import cProfile
cProfile.run('test_tagged = test["phrases"].apply(postag)')

In [ ]:
translat = translations[0:100]

In [ ]:
tags_DF

In [ ]:
# Now transform all the data
import pandas as pd
import numpy as np
from post_tag import *
import csv
from time import time
translations = pd.read_csv("training.txt", sep="\t", header=None, names=['labels','phrases'], quoting=csv.QUOTE_NONE)
fdist = itertools.permutations(words, 2)
permu_names = [f for f in fdist]



In [ ]:
trans = translat['phrases'].apply(postag)
tags_DF = pd.DataFrame(np.asmatrix(list(trans)),columns = permu_names)

test_new = pd.concat([translat, tags_DF], axis=1,join_axes=[translat.index])
test_new.drop('phrases',axis=1, inplace=True)
test_new.to_csv(path_or_buf='pos_tags_train_data.csv')